## Step 0: Importing modules

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np

## Step 1: Calculating Risk (Variance/Standard Deviation)

In statistics, **variance** is a measurement of how far each number in a data set is from the mean (average), and thus from every other number in the set. This can be thought of how much we can expect values in the data set to differ from eachother. 

**Standard Deviation** is the square root of the variance, meaning it is expressed in the same units as the original data (variance is expressed in squared units).

In quantitative finance, **volatility** is used as a statistical measure of risk in an asset's returns. It is often measured from either the standard deviation or variance in returns.

In the context of quantitative finance, the **volatility of rrturns** is used as a measure of **risk**. With higher volatility, the returns of an asset at different times vary more, and vice-versa. As such, higher volatility is associated with a greater possibility for larger gains or losses, and vice versa. Thus, high volatility in a stock's returns is associated with higher risk along with higher return.

Let's calculate the volatility of a stock (using standard deviation):

In [ ]:
stock_ticker = "MSFT"  # Microsoft
data = yf.download(stock_ticker, start="2024-01-01", end="2025-01-03")  # Get last year's daily price data for MSFT

First, we need to know the daily returns.

In [ ]:
data["Pct Return"] = data['Close'].pct_change()  # Calculate daily percent return
data.dropna(inplace=True)  # Drop the first row since it will have a NaN value
data["Pct Return"]

Now we can take the standard deviation of these returns to get the *daily* volatility over the given timeframe.

In [ ]:
daily_vol = data["Pct Return"].std()  # Calculate daily volatility
print(daily_vol)


While we are at it, let's get the average daily return

In [ ]:
daily_mean_returns = data["Pct Return"].mean()  # Calculate daily mean returns
print(daily_mean_returns)

## Step 2: Annualizing Returns and Volatility

Right now, our returns and volatility are measured on a daily time frame. Let's get the returns and volatility **annualized** over the year.

Since there are 252 *trading days* in a year (typically, RIP Jimmy Carter), we need to multiply the average daily return by 252. However, since standard deviation is the *square root* of variance, we need to multiply the variance by the *square root of 252*.

If we had monthly returns and volatility, we would similarly multiply them by 12 and root(12), since there are 12 months of trading in a year, etc.

In [ ]:
print(data["Close"].shape)  # Confirming there are 252 trading days in our timeframe.

In [ ]:
mean_returns_annualized = daily_mean_returns * 252  # Annualize mean returns
volatility_annualized = daily_vol * np.sqrt(252)  # Annualize volatility
print(mean_returns_annualized, volatility_annualized)

## Step 3: Calculating Risk Free Rate, Excess Returns, and Sharpe Ratio

Let's use U.S. T-bills (13 week) as the risk free asset. **^IRX** tracks the returns of these, so we can get that data to calculate our risk free rate.

In [ ]:
tbill_data = yf.download("^IRX", start="2024-01-01", end="2025-01-03")  # Get last year's T-bill data

NOTE: These "Close" values are the **ANNUALIZED YIELDS** of the 13-week T-bill, which means we can use these numbers directly as the percentage return an investor would recieve by buying them on that date and holding until maturity (repeated) for a year. 

Let's get the yield for the start of 2024. (We are simplifying a bit)

In [ ]:
risk_free_rate = float(tbill_data['Close'].iloc[0]) / 100 # Get the risk-free rate from the first row, scaled
risk_free_rate

Now we can calculate MSFT's excess annualized returns:

In [ ]:
excess_annualized_return = mean_returns_annualized - risk_free_rate  # Calculate excess returns
print(excess_annualized_return)

Finally, we can now calculate the sharpe ratio, which is the annualized excess returns divided by the annualized volatility:

In [ ]:
sharpe = excess_annualized_return / volatility_annualized  # Calculate Sharpe ratio
sharpe

## Step 4: Calculating Other Performance Metrict (MDD, Sortino, and VaR)

### Step 4.1: Max Drawdowns

First, let's calculate the **drawdowns** (movements from a peak to a through) of MSFT during the year on a rolling basis. To do this, we first need to calculate the rolling cummulative returns (returns from start to a given date).

In [ ]:
cum_returns = (1 + data["Pct Return"]).cumprod()  # Calculate cumulative returns
cum_max = cum_returns.cummax()  # Calculate running maximum returns for calculating drawdowns
drawdown = (cum_returns - cum_max) / cum_max  # Calculate drawdowns on a rolling basis

drawdown.plot()  # Plot drawdowns

Above we can see the drawdowns at different points in the year. The **Max Drawdown** in our yearly period is simply the worst of these. This is the worst-case possible timing of investing (long) in MSFT. Visually you can see this would be buying around July '24 and selling August '24.

In [ ]:
max_drawdown = drawdown.min()  # Calculate maximum drawdown
print(max_drawdown)

### Step 4.2: Sortino Ratio

**Sortino Ratio** is a lot like the sharpe ratio, but only considering standard deviation of *negative* asset returns (since big sharpe jumps upwards in returns are generally better welcomed :)). 

In [ ]:
downside_deviation = data["Pct Return"][data["Pct Return"] < 0].std()  # Calculate downside deviation
annualized_downside_deviation = downside_deviation * np.sqrt(252)  # Annualize downside deviation
sortino = excess_annualized_return / annualized_downside_deviation  # Calculate Sortino ratio
print(sortino)

### Step 4.3: Value at Risk (VaR)

To calculate Value at Risk, we simply take the 5th quantile of our historic returns. This gives us the "minimum worst 5% of daily returns" we can assume in the future, *assuming that historic returns predict future returns and returns are normally distributed*.

In [ ]:
var = data["Pct Return"].quantile(0.05) # Calculate VaR
print(var)

Here we see that we can expect to lose at least 2.1% on the worst 5% of days.

### Step 4.4: Cummulative Value at Risk (CVaR)

Cummulative Value at Risk is simply the mean of the fifth quantile of historic returns. This is more like the expected value of our returns given that the returns are within the 5% worst.

In [ ]:
cvar = data["Pct Return"][data["Pct Return"] <= var].mean()  # Calculate CVaR
print(cvar)